In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

In [48]:
hkd_thb = pd.read_csv("hkdthb.csv")
hkd_thb.drop(hkd_thb.tail(1).index,inplace=True)
hkd_thb["Time"] = pd.to_datetime(hkd_thb["Time"])
hkd_thb.tail()

,Time,Open,High,Low,Last,Change,%Chg,Volume
514,2020-01-24,3.9199,3.9350,3.9166,3.9271,0.0072,+0.18%,1481.0
515,2020-01-23,3.9058,3.9288,3.9020,3.9199,0.0141,+0.36%,1569.0
516,2020-01-22,3.9098,3.9105,3.9008,3.9058,-0.0041,-0.10%,1652.0
517,2020-01-21,3.9022,3.9123,3.8971,3.9099,0.0078,+0.20%,1617.0
518,2020-01-20,3.9135,3.9135,3.8480,3.9021,-0.0046,-0.12%,1692.0


In [49]:
data = [go.Candlestick(x=hkd_thb['Time'],
                       open=hkd_thb["Open"],
                       high=hkd_thb['High'],
                       low=hkd_thb['Low'],
                       close=hkd_thb['Last'])]
layout = go.Layout(
    autosize=True,
    width=1000,
    height=1000,
    yaxis= go.layout.YAxis(autorange = True,fixedrange= False, range=[3, 5])
)
fig = go.Figure(data = data, layout = layout)

fig.show()

In [50]:
#preprocessing
scaler = MinMaxScaler()
scaler.fit(hkd_thb[["Open","High","Low","Last"]])
np_array = scaler.transform(hkd_thb[["Open","High","Low","Last"]])
print(np_array)

[[0.81546708 0.82360923 0.81878354 0.76846537]
 [0.81546708 0.81042838 0.84758497 0.80960582]
 [0.82312404 0.81798798 0.85008945 0.8155377 ]
 ...
 [0.14184533 0.12211669 0.21717352 0.13451971]
 [0.12729709 0.12560574 0.21055456 0.1423651 ]
 [0.14892802 0.12793177 0.12271914 0.12743972]]


In [51]:
preprocessed = pd.DataFrame(np_array,columns=["Open","High","Low","Last"])
preprocessed["Time"] = hkd_thb["Time"]
preprocessed

,Open,High,Low,Last,Time
0,0.815467,0.823609,0.818784,0.768465,2022-01-17
1,0.815467,0.810428,0.847585,0.809606,2022-01-14
2,0.823124,0.817988,0.850089,0.815538,2022-01-13
3,0.824081,0.851328,0.860644,0.823192,2022-01-12
4,0.905054,0.895522,0.869052,0.823192,2022-01-11
...,...,...,...,...,...
514,0.161179,0.169607,0.245438,0.175277,2020-01-24
515,0.134188,0.157589,0.219320,0.161500,2020-01-23
516,0.141845,0.122117,0.217174,0.134520,2020-01-22
517,0.127297,0.125606,0.210555,0.142365,2020-01-21


In [52]:
data = [go.Candlestick(x=preprocessed['Time'],
                       open=preprocessed["Open"],
                       high=preprocessed['High'],
                       low=preprocessed['Low'],
                       close=preprocessed['Last'])]
layout = go.Layout(
    autosize=True,
    width=1000,
    height=1000,
    yaxis= go.layout.YAxis(autorange = True,fixedrange= False, range=[3, 5])
)
fig = go.Figure(data = data, layout = layout)

fig.show()

In [53]:
training_data = preprocessed.iloc[:-100]
testing_data = preprocessed.iloc[-100:]

In [54]:
training_data

,Open,High,Low,Last,Time
0,0.815467,0.823609,0.818784,0.768465,2022-01-17
1,0.815467,0.810428,0.847585,0.809606,2022-01-14
2,0.823124,0.817988,0.850089,0.815538,2022-01-13
3,0.824081,0.851328,0.860644,0.823192,2022-01-12
4,0.905054,0.895522,0.869052,0.823192,2022-01-11
...,...,...,...,...,...
414,0.320253,0.336887,0.367442,0.298890,2020-06-12
415,0.331164,0.319248,0.352952,0.320704,2020-06-11
416,0.382466,0.372165,0.405725,0.331420,2020-06-10
417,0.405628,0.391743,0.456887,0.382893,2020-06-09


In [55]:
testing_data

,Open,High,Low,Last,Time
419,0.437979,0.426439,0.489088,0.424990,2020-06-05
420,0.445827,0.453189,0.508766,0.437620,2020-06-04
421,0.437021,0.441558,0.502147,0.446230,2020-06-03
422,0.466501,0.453964,0.504651,0.436854,2020-06-02
423,0.499809,0.483233,0.533453,0.466705,2020-06-01
...,...,...,...,...,...
514,0.161179,0.169607,0.245438,0.175277,2020-01-24
515,0.134188,0.157589,0.219320,0.161500,2020-01-23
516,0.141845,0.122117,0.217174,0.134520,2020-01-22
517,0.127297,0.125606,0.210555,0.142365,2020-01-21


In [56]:
class LSTM(nn.Module):
    def __init__(self,num_hidden = 50):
        super(LSTM, self).__init__()
        self.num_hidden = num_hidden
        self.lstm1 = nn.LSTMCell(4 , self.num_hidden)
        self.lstm2 = nn.LSTMCell(self.num_hidden , self.num_hidden)
        self.linear = nn.Linear(self.num_hidden, 4)
    
    def forward(self, x, future = 0):
        outputs = []
        num_samples = x.size(0)
        h_t = torch.zeros(num_samples, self.num_hidden, dtype=torch.double)
        c_t = torch.zeros(input.size(0), self.num_hidden, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), self.num_hidden, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), self.num_hidden, dtype=torch.double)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [57]:
torch.manual_seed(0)
model = LSTM()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)